In [ ]:
# Concatenate X_train and y_train
train_data = pd.concat([X_train, y_train], axis=1)

# Create a copy of the DataFrame
train_data_copy = train_data.copy() 

features_copy = train_data_copy.select_dtypes(include=[np.number]).columns.tolist()

if 'popularity' in features_copy:
    features_copy.remove('popularity')

for x in features_copy:
    q75, q25 = np.percentile(train_data_copy.loc[:,x],[75,25])
    intr_qr = q75-q25

    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)

    # Replace data points that lie outside of the lower and the upper bound with a NULL value.
    train_data_copy.loc[train_data_copy[x] < min,x] = np.nan
    train_data_copy.loc[train_data_copy[x] > max,x] = np.nan

# Check the sum of null values or missing values
print(train_data_copy.isnull().sum())

In [ ]:
# Remove rows with np.nan values (outliers)
train_data_copy = train_data_copy.dropna()

In [ ]:
print(train_data_copy.isnull().sum())

In [ ]:
# Separate X_train_copy and y_train again
X_train_copy = train_data_copy.drop(columns=[y_train.name])
y_train_copy = train_data_copy[y_train.name]

In [ ]:
print(X_train_copy.isna().sum())

In [ ]:
print(y_train_copy.isna().sum())

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

#from sklearn.ensemble import VotingClassifier

# Define the pipelines
dtr_pipe = make_pipeline_imb(SMOTE(random_state=42), preprocessing, DecisionTreeClassifier())
rf_pipe = make_pipeline_imb(SMOTE(random_state=42), preprocessing, RandomForestClassifier())
brf_pipe = make_pipeline_imb(SMOTE(random_state=42), preprocessing, BalancedRandomForestClassifier(n_estimators=50, sampling_strategy="all", replacement=True, bootstrap=False, random_state=42))


In [ ]:
train_score = hbgb_clf_pipe.score(X_train, y_train)
print(f'Training score: {train_score}')


test_score = hbgb_clf_pipe.score(X_test, y_test)
print(f'Test score: {test_score}')

In [ ]:
y_train_pred_hbgb = cross_val_predict(hbgb_clf_pipe, X_train, y_train, cv=5)

plt.rc('font', size=9)  # extra code – make the text smaller

# Display the confusion matrix
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred_hbgb)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate classification metrics
accuracy = accuracy_score(y_train, y_train_pred_hbgb)
precision = precision_score(y_train, y_train_pred_hbgb)
recall = recall_score(y_train, y_train_pred_hbgb)
f1 = f1_score(y_train, y_train_pred_hbgb)

# Print the classification metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_pipe = make_pipeline_imb(SMOTE(random_state=42), preprocessing, AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), algorithm='SAMME', learning_rate=1, random_state=42))

ada_pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate classification metrics
accuracy = accuracy_score(y_train, y_train_pred_ada)
precision = precision_score(y_train, y_train_pred_ada)
recall = recall_score(y_train, y_train_pred_ada)
f1 = f1_score(y_train, y_train_pred_ada)

# Print the classification metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')